In [12]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from SVM import accuracy
from scaler import scaler 
from subGD import subGD

Xtrain = pd.read_csv("MNIST_X_train.csv")
ytrain = pd.read_csv("MNIST_Y_train.csv")
Xtest = pd.read_csv("MNIST_X_test.csv")
ytest = pd.read_csv("MNIST_Y_test.csv")
npXtrain = Xtrain.to_numpy()
npytrain = ytrain.to_numpy()
npXtest = Xtest.to_numpy()
npytest = ytest.to_numpy()
print("The shape of Xtrain is {}".format(Xtrain.shape))
print("The shape of Xtest is {}".format(Xtest.shape))

# Feature scaling
mean = np.mean(npXtrain, axis=0).reshape((1, npXtrain.shape[1]))
std = np.std(npXtrain, axis=0).reshape((1, npXtrain.shape[1]))
scaled_Xtrain = scaler(npXtrain, mean, std)
scaled_Xtest = scaler(npXtest, mean, std)
m1 = npXtrain.shape[0]
m2 = npXtest.shape[0]
newXtrain = np.hstack((np.ones((m1,1)), scaled_Xtrain))
newXtest = np.hstack((np.ones((m2,1)), scaled_Xtest))



The shape of Xtrain is (2000, 784)
The shape of Xtest is (500, 784)


In [14]:
iterations = 100
alpha = 0.03
lam = 1/400

# Extract samples corresponding to each class
dataList = [newXtrain[np.where(npytrain == i)[0]] for i in range(10)]       
# The elements of paraMat are parameters returned by simple_smo_poly
paraMat = np.zeros((10,10), dtype=object)
# one vs one approach
for i in range(10):
    for j in range(10):
        if j > i:
            targetI = np.ones(dataList[i].shape[0]).reshape((-1,1))
            targetJ = -np.ones(dataList[j].shape[0]).reshape((-1,1))
            data = np.vstack((dataList[i], dataList[j]))
            target = np.vstack((targetI, targetJ))
            # Train class i vs class j
            paraMat[i,j] = subGD(data, target, iterations, alpha, lam)
            # compute training accuracy
            predLabelsIJ = np.zeros((target.shape[0],1))
            for ind in np.arange(target.shape[0]):
                if data[ind,:]@paraMat[i,j] >= 0:
                    predLabelsIJ[ind] = 1
                else:
                    predLabelsIJ[ind] = -1
            scoreIJ = accuracy(target, predLabelsIJ)
            print("Training class {} vs class {} is complete. The training accuracy is {:.2f}%".format(i,j,scoreIJ*100))

predLabels = np.zeros(npytest.shape[0]).reshape((-1, 1))
# one vs one classification
for ind in np.arange(npytest.shape[0]):
    OvOlabel_list = np.zeros(10)
    for i in range(9):
        for j in range(10):
            if j > i:
                if newXtest[ind,:]@paraMat[i,j] >= 0:
                    OvOlabel_list[i] += 1
                else:
                    OvOlabel_list[j] += 1
    # end for
    predLabels[ind, 0] = np.argmax(OvOlabel_list) 

score = accuracy(npytest, predLabels)
print("The accuracy of multiclass classification is {:.2f}%".format(score*100))  

Training class 0 vs class 1 is complete. The training accuracy is 99.77%
Training class 0 vs class 2 is complete. The training accuracy is 98.33%
Training class 0 vs class 3 is complete. The training accuracy is 97.32%
Training class 0 vs class 4 is complete. The training accuracy is 99.51%
Training class 0 vs class 5 is complete. The training accuracy is 97.69%
Training class 0 vs class 6 is complete. The training accuracy is 99.50%
Training class 0 vs class 7 is complete. The training accuracy is 99.76%
Training class 0 vs class 8 is complete. The training accuracy is 97.35%
Training class 0 vs class 9 is complete. The training accuracy is 98.72%
Training class 1 vs class 2 is complete. The training accuracy is 96.44%
Training class 1 vs class 3 is complete. The training accuracy is 95.68%
Training class 1 vs class 4 is complete. The training accuracy is 99.31%
Training class 1 vs class 5 is complete. The training accuracy is 98.33%
Training class 1 vs class 6 is complete. The traini